<a href="https://colab.research.google.com/github/abdullahui/Final-Exam-KLBD/blob/main/Ch2_Memory_basedCollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2: Memory-based Collaborative Filtering

## Introduction

Chapter two talks about Memory-based Collaborative Filtering. There is two main algorithms to do it:
1. User-based (or user to user) Collaborative Filtering : implements user-based collaborative filtering.
2. Item-based (or item to item) Collaborative Filtering : implements item-based collaborative filtering.


Memory based collaborative filtering (CF) also known as nearest neighbors based CF makes recommendation based on similar behavious of users and items. There are two types of memory based CF : <b>user-based</b> and <b>item-based</b> CF. Both of these algorithm usually proceed in three stages :

1. Similarity computation (between users or items)
2. Rating prediction (using ratings of similar users or items)
3. Top-N recommendation

we will explain each algorithm in the next two sections.

## Section 1: User-based Collaborative Filtering

**Idea**

Let $u$ be the user for which we plan to make recommendations. 

1. Find other users whose past rating behavior is similar to that of $u$
2. Use their ratings on other items to predict what the current user will like

**Algorithm:** user-to-user collaborative filtering
1. Identify $G_u$, the set of $k$ users similar to an active user $u$
2. Find candidate items
3. Rating prediction
4. Top-N recommendation

### First we prepare the tool that we will use it.

#### Download tools

In [ ]:
import os

if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

--2023-01-01 11:17:14--  https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip [following]
--2023-01-01 11:17:14--  https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15312323 (15M) [application/zip]
Saving to: ‘recsys.zip’

recsys.zip          100%[===================>]  14.60M  --.-KB/s    in 0.1s    

2023-01-01 11:17:16 (111 MB/s) - ‘recsys.zip’ saved

#### Import requirements

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from recsys.datasets import ml100k
from recsys.preprocessing import ids_encoder

import pandas as pd
import numpy as np
import zipfile

#### Load MovieLen ratings

In [ ]:
ratings, movies = ml100k.load()

Download data 100.2%
Successfully downloaded ml-100k.zip 4924029 bytes.
Unzipping the ml-100k.zip zip file ...


#### userids and itemids encoding

this create the encoder

In [ ]:
ratings, uencoder, iencoder = ids_encoder(ratings)

#### Transform rating dataframe to matrix

In [ ]:
def ratings_matrix(ratings):    
    return csr_matrix(pd.crosstab(ratings.userid, ratings.itemid, ratings.rating, aggfunc=sum).fillna(0).values)    

R = ratings_matrix(ratings)

### Algorithm Steps

#### **Step 1. Identify $G_u$, the set of $k$ users similar to an active user $u$** 

To find the $k$ most similar users to $u$, we use the cosine similarity and compute $w_{u,v}$ for all $v\in U$. Fortunately, libraries such as <i>scikit-learn (sklearn)</i> are very useful for such tasks :

1. First of all, we create a nearest neighbors model with sklearn through the function ```create_model()```. This function creates and fit a nearest neighbors model with user's ratings. We can choose ```cosine``` or ```euclidian``` based similarity metric. ```n_neighbors=21``` define the number of neighbors to return. With $k=20$ neighbors, $|G_u|=21$ as $G_u$ contains $20$ similar users added to the active user $u$. That is why ```n_neighbors=21```. Each row $r_u$ of the rating matrix $R$ represents ratings of user $u$ on all items of the database. Missing ratings are replaced with $0.0$. 
```python
R[u,:] # uth row of the rating matrix R. Ratings of user u on all items in the database
```
2. Function ```nearest_neighbors()``` returns the knn users for each user.

In [ ]:
def create_model(rating_matrix, metric):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm='brute')
    model.fit(rating_matrix)    
    return model

In [ ]:
def nearest_neighbors(rating_matrix, model):
    """    
    :param rating_matrix : rating matrix of shape (nb_users, nb_items)
    :param model : nearest neighbors model    
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

Now, Let's call functions ```create_model()``` and ```nearest_neighbors()``` to respectively create the $k$-NN model and compute the nearest neighbors for a given user

In [ ]:
model = create_model(rating_matrix=R, metric='cosine') # we can also use the 'euclidian' distance
similarities, neighbors = nearest_neighbors(R, model)

#### **Step 2. Find candidate items**


The set $C$ of candidate items are the most frequent ones purchased by users in $G_u$ for an active user $u$ and not purchased by $u$.

Function ```find_candidate_items()``` : find items purchased by these similar users as well as their frequency. Note that the frequency of the items in the set $C$ can be computed by just counting the actual occurrence frequency of that items.

1. ```Gu_items``` : frequent items of $G_u$ in decreasing order of frequency.
2. ```active_items``` : items already purchased by the active user
3. ```candidates``` : frequent items of $G_u$ not purchased by the active user $u$

In [ ]:
def find_candidate_items(userid):
    """
    Find candidate items for an active user
    
    :param userid : active user
    :param neighbors : users similar to the active user        
    :return candidates : top 30 of candidate items
    """
    user_neighbors = neighbors[userid]
    activities = ratings.loc[ratings.userid.isin(user_neighbors)]
    
    # sort items in decreasing order of frequency
    frequency = activities.groupby('itemid')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.itemid
    active_items = ratings.loc[ratings.userid == userid].itemid.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]
        
    return candidates

#### **Step 3. Rating prediction**

Now it's time to predict what score the active user $u$ would have given to each of the top-30 candidate items.

To predict the score of $u$ on a candidate item $i$ ,we need :
1. Similarities between $u$ and all his neighbors $v \in G_u$ who rated item $i$ : function ```nearest_neighbors()``` returns similar users of a user as well as their corresponding similarities.
2. Normalized ratings of all $v \in G_u$ on item $i$. The normalized rating of user $v$ on item $i$ is defined by $r_{v,i}-\bar{r}_v$.

Next, let's compute the mean rating of each user and the normalized ratings for each item. The DataFrame ```mean``` contains mean rating for each user. With the mean rating of each user, we can add an extra column ```norm_rating``` to the ```ratings```'s DataFrame which can be accessed to make predictions.

In [ ]:
# mean ratings for each user
mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')

# normalized ratings for each items
mean_ratings['norm_rating'] = mean_ratings['rating'] - mean_ratings['rating_mean']

mean = mean.to_numpy()[:, 1]

In [ ]:
np_ratings = mean_ratings.to_numpy()

Let us define function ```predict``` that predict rating between user $u$ and 

1.   List item
2.   List item

In [ ]:
def predict(userid, itemid):
    """
    predict what score userid would have given to itemid.
    
    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction
        
    :return
        - r_hat : predicted rating of user userid on item itemid
    """
    user_similarities = similarities[userid]
    user_neighbors = neighbors[userid]
    # get mean rating of user userid
    user_mean = mean[userid]
    
    # find users who rated item 'itemid'
    iratings = np_ratings[np_ratings[:, 1].astype('int') == itemid]
    
    # find similar users to 'userid' who rated item 'itemid'
    suri = iratings[np.isin(iratings[:, 0], user_neighbors)]
    
    # similar users who rated current item (surci)
    normalized_ratings = suri[:,4]
    indexes = [np.where(user_neighbors == uid)[0][0] for uid in suri[:, 0].astype('int')]
    sims = user_similarities[indexes]
    
    num = np.dot(normalized_ratings, sims)
    den = np.sum(np.abs(sims))
    
    if num == 0 or den == 0:
        return user_mean
    
    r_hat = user_mean + np.dot(normalized_ratings, sims) / np.sum(np.abs(sims))
    
    return r_hat

Now, we can make rating prediction for a given user on each item in his set of candidate items.

In [ ]:
def user2userPredictions(userid, pred_path):
    """
    Make rating prediction for the active user on each candidate item and save in file prediction.csv
    
    :param
        - userid : id of the active user
        - pred_path : where to save predictions
    """    
    # find candidate items for the active user
    candidates = find_candidate_items(userid)
    
    # loop over candidates items to make predictions
    for itemid in candidates:
        
        # prediction for userid on itemid
        r_hat = predict(userid, itemid)
        
        # save predictions
        with open(pred_path, 'a+') as file:
            line = '{},{},{}\n'.format(userid, itemid, r_hat)
            file.write(line)

In [ ]:
import sys

def user2userCF():
    """
    Make predictions for each user in the database.    
    """
    # get list of users in the database
    users = ratings.userid.unique()
    
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
    
    saved_predictions = 'predictions.csv'    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    for count, userid in enumerate(users):        
        # make rating predictions for the current user
        user2userPredictions(userid, saved_predictions)
        _progress(count)

In [ ]:
user2userCF()

Rating predictions. Progress status : 99.9%

As we see here, the progress of rating predictions is so satisfying.

#### **Step 4. Top-N recommendation**


Function ```user2userRecommendation()``` reads predictions for a given user and return the list of items in decreasing order of predicted rating.

In [ ]:
def user2userRecommendation(userid):
    """
    """
    # encode the userid
    uid = uencoder.transform([userid])[0]
    saved_predictions = 'predictions.csv'
    
    predictions = pd.read_csv(saved_predictions, sep=',', names=['userid', 'itemid', 'predicted_rating'])
    predictions = predictions[predictions.userid==uid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)
    
    List.userid = uencoder.inverse_transform(List.userid.tolist())
    List.itemid = iencoder.inverse_transform(List.itemid.tolist())
    
    List = pd.merge(List, movies, on='itemid', how='inner')
    
    return List

In [ ]:
user2userRecommendation(212)

,userid,itemid,predicted_rating,title
0,212,483,4.871495,Casablanca (1942)
1,212,357,4.764547,One Flew Over the Cuckoo's Nest (1975)
2,212,50,4.660002,Star Wars (1977)
3,212,98,4.613636,"Silence of the Lambs, The (1991)"
4,212,64,4.550733,"Shawshank Redemption, The (1994)"
5,212,194,4.522336,"Sting, The (1973)"
6,212,174,4.521300,Raiders of the Lost Ark (1981)
7,212,134,4.414819,Citizen Kane (1941)
8,212,187,4.344531,"Godfather: Part II, The (1974)"
9,212,196,4.303696,Dead Poets Society (1989)


So, the rating predection for the user that have ID (212) is between [3, 4.9].

#### **Evaluation with Mean Absolute Error (MAE)**

First we will evaluate the model on test data, then we calculate the mean absolute error (**MAE**) to evaluate the performance of a predictive model

In [ ]:
from recsys.preprocessing import train_test_split, get_examples

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

def evaluate(x_test, y_test):
    print('Evaluate the model on {} test data ...'.format(x_test.shape[0]))
    preds = list(predict(u,i) for (u,i) in x_test)
    mae = np.sum(np.absolute(y_test - np.array(preds))) / x_test.shape[0]
    print('\nMAE :', mae)
    return mae

In [ ]:
evaluate(x_test, y_test)

Evaluate the model on 10000 test data ...

MAE : 0.7505910931068639


0.7505910931068639

This means that on average, the model's predictions are off by 0.7505910931068639 units.

## Section 2: Item-based Collaborative Filtering

**Idea**

Let $u$ be the active user and $i$ the referenced item
1. If $u$ liked items similar to $i$, he will probably like item $i$.
2. If he hated or disliked items similar to $i$, he will also hate item $i$.

The idea is therefore to look at how an active user $u$ rated items similar to $i$ to know how he would have rated item $i$

**Algorithm:** item-to-item collaborative filtering
1. Find similarities for each of the items
2. Top N recommendation for a given user

   a- Finding candidate items

   b- Find similarity between each candidate item and the set $I_u$

   c- Rank candidate items according to their similarities to $I_u$


### First we prepare the tool that we will use it.

#### Import useful requirements

In [ ]:
import os

if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

#### Import requirements

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from recsys.datasets import ml1m, ml100k
from recsys.preprocessing import ids_encoder

import pandas as pd
import numpy as np
import os
import sys

#### Load ratings

In [ ]:
ratings, movies = ml100k.load()

#### userids and itemids encoding

In [ ]:
# create the encoder
ratings, uencoder, iencoder = ids_encoder(ratings)

Let's now implements the item-based collaborative filtering algorithm described above

### Algorithm Steps

#### **Step 1. Find similarities for each of the items**

To compute similarity between two items $i$ and $j$, we need to :

1. find all users who rated both of them,
2. Normalize their ratings on items $i$ and $j$
3. Apply the cosine metric to the normalized ratings to compute similarity between $i$ and $j$

Function ```normalize()``` process the rating dataframe to normalize ratings of all users

In [ ]:
def normalize():
    # compute mean rating for each user
    mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')
    
    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']
    return mean.to_numpy()[:, 1], norm_ratings

In [ ]:
mean, norm_ratings = normalize()
np_ratings = norm_ratings.to_numpy()
norm_ratings.head()

,userid,itemid,rating,rating_mean,norm_rating
0,0,0,5,3.610294,1.389706
1,0,1,3,3.610294,-0.610294
2,0,2,4,3.610294,0.389706
3,0,3,3,3.610294,-0.610294
4,0,4,3,3.610294,-0.610294


now that each rating has been normalized, we can represent each item by a vector of its normalized ratings

In [ ]:
def item_representation(ratings):    
    return csr_matrix(
        pd.crosstab(ratings.itemid, ratings.userid, ratings.norm_rating, aggfunc=sum).fillna(0).values
    )

In [ ]:
R = item_representation(norm_ratings)

Let's build and fit our $k$-NN model using sklearn

In [ ]:
def create_model(rating_matrix, k=20, metric="cosine"):
    """
    :param R : numpy array of item representations
    :param k : number of nearest neighbors to return    
    :return model : our knn model
    """    
    model = NearestNeighbors(metric=metric, n_neighbors=k+1, algorithm='brute')
    model.fit(rating_matrix)    
    return model

##### **Similarities computation**

Similarities between items can be measured with the *Cosine* or *Eucliedian* distance. The ***NearestNeighbors*** class from the sklearn library simplifies the computation of neighbors. We just need to specify the metric (e.g. cosine or euclidian) that will be used to compute similarities.

The above method, ```create_model```, creates the kNN model and the following ```nearest_neighbors``` method uses the created model to kNN items. It returns nearest neighbors as well as similarities measures for each items.

```nearest_neighbors``` returns :
- ```similarities``` : numpy array of shape $(n,k)$
- ```neighbors``` : numpy array of shape $(n,k)$

where $n$ is the total number of items and $k$ is the number of neighbors to return, specified when creating the kNN model.

In [ ]:
def nearest_neighbors(rating_matrix, model):
    """
    compute the top n similar items for each item.    
    :param rating_matrix : items representations
    :param model : nearest neighbors model    
    :return similarities, neighbors
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)    
    return similarities[:,1:], neighbors[:,1:]

##### **Ajusted Cosine Similarity**

In the context of item-based collaborative filtering, the adjusted cosine similarity has shown to be more efficient that the cosine or the euclidian distance.
We will implement it with the method ```adjusted_cosine```, with some helper function :

- ```save_similarities``` : since the computation of the adjusted cosine similarity is time consuming, around 5 mins for the ml100k dataset, we use this method to save the computed similarities for lated usage.
- ```load_similarities``` : load the saved similarities
- ```cosine``` : cosine distance between two vectors.

In [ ]:
def save_similarities(similarities, neighbors, dataset_name):    
    base_dir = 'recsys/weights/item2item'
    save_dir = os.path.join(base_dir, dataset_name)
    os.makedirs(save_dir, exist_ok=True)    
    similarities_file_name = os.path.join(save_dir, 'similarities.npy')
    neighbors_file_name = os.path.join(save_dir, 'neighbors.npy')    
    try:
        np.save(similarities_file_name, similarities)
        np.save(neighbors_file_name, neighbors)        
    except ValueError as error:
        print(f"An error occured when saving similarities, due to : \n ValueError : {error}")

        
def load_similarities(dataset_name, k=20):
    base_dir = 'recsys/weights/item2item'
    save_dir = os.path.join(base_dir, dataset_name)    
    similiraties_file = os.path.join(save_dir, 'similarities.npy')
    neighbors_file = os.path.join(save_dir, 'neighbors.npy')    
    similarities = np.load(similiraties_file)
    neighbors = np.load(neighbors_file)    
    return similarities[:,:k], neighbors[:,:k]


def cosine(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


def adjusted_cosine(np_ratings, nb_items, dataset_name):
    similarities = np.zeros(shape=(nb_items, nb_items))
    similarities.fill(-1)
    
    def _progress(count):
        sys.stdout.write('\rComputing similarities. Progress status : %.1f%%' % (float(count / nb_items)*100.0))
        sys.stdout.flush()
        
    items = sorted(ratings.itemid.unique())    
    for i in items[:-1]:
        for j in items[i+1:]:            
            scores = np_ratings[(np_ratings[:, 1] == i) | (np_ratings[:, 1] == j), :]
            vals, count = np.unique(scores[:,0], return_counts = True)
            scores = scores[np.isin(scores[:,0], vals[count > 1]),:]

            if scores.shape[0] > 2:
                x = scores[scores[:, 1].astype('int') == i, 4]
                y = scores[scores[:, 1].astype('int') == j, 4]
                w = cosine(x, y)

                similarities[i, j] = w
                similarities[j, i] = w
        _progress(i)
    _progress(nb_items)
    
    # get neighbors by their neighbors in decreasing order of similarities
    neighbors = np.flip(np.argsort(similarities), axis=1)
    
    # sort similarities in decreasing order
    similarities = np.flip(np.sort(similarities), axis=1)
    
    # save similarities to disk
    save_similarities(similarities, neighbors, dataset_name=dataset_name) 
    
    return similarities, neighbors

now, we can call the ```adjusted_cosine``` function to compute and save items similarities and neighbors based on the adjusted cosine metric. 

uncomment the two lines of the following cell to compute the adjusted cosine between all items. As we have already run the next cell before, we will just load the precomputed similarities for further use.

In [ ]:
# nb_items = ratings.itemid.nunique()
# similarities, neighbors = adjusted_cosine(np_ratings, nb_items=nb_items, dataset_name='ml100k')

Among the following similarity metrics, choose the one you wish to use for the item-based collaborative filtering :

- **euclidian** or **cosine** : choose *euclidian* or *cosine* to initialise the similarity model through the sklearn library.
- **adjusted_cosine** : choose the *adjusted_cosine* metric to load similarities computed and saved through the ```adjusted_cosine``` function.

In this case, we will use the *adjusted_cosine* metric.

In [ ]:
# metric : choose among [cosine, euclidean, adjusted_cosine]

metric = 'adjusted_cosine'

if metric == 'adjusted_cosine':
    similarities, neighbors = load_similarities('ml100k')
else:
    model = create_model(R, k=21, metric=metric)
    similarities, neighbors = nearest_neighbors(R, model)

In [ ]:
print('neighbors shape : ', neighbors.shape)
print('similarities shape : ', similarities.shape)

neighbors shape :  (1682, 20)
similarities shape :  (1682, 20)


```neighbors``` and ```similarities``` are numpy array, were each entries are list of 20 neighbors with their corresponding similarities

#### **Step 2. Top N recommendation for a given user**

Top-N recommendations are made for example for a user $u$ who has already rated a set of items $I_u$

##### **2.a- Finding candidate items**

To find candidate items for user $u$, we need to :

1. Find the set $I_u$ of items already rated by user $u$,
2. Take the union of similar items as $C$ for all items in $I_u$
3. exclude from the set $C$ all items in $I_u$, to avoid recommend to a user items he has already purchased.

These are done in function ```candidate_items()```

In [ ]:
def candidate_items(userid):
    """
    :param userid : user id for which we wish to find candidate items    
    :return : I_u, candidates
    """
    
    # 1. Finding the set I_u of items already rated by user userid
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1].astype('int')
    
    # 2. Taking the union of similar items for all items in I_u to form the set of candidate items
    c = set()
        
    for iid in I_u:    
        # add the neighbors of item iid in the set of candidate items
        c.update(neighbors[iid])
        
    c = list(c)
    # 3. exclude from the set C all items in I_u.
    candidates = np.setdiff1d(c, I_u, assume_unique=True)
    
    return I_u, candidates

In [ ]:
test_user = uencoder.transform([1])[0]
i_u, u_candidates = candidate_items(test_user)

In [ ]:
print('number of items purchased by user 1 : ', len(i_u))
print('number of candidate items for user 1 : ', len(u_candidates))

number of items purchased by user 1 :  272
number of candidate items for user 1 :  893


##### **2.b- Find similarity between each candidate item and the set $I_u$**

In [ ]:
def similarity_with_Iu(c, I_u):
    """
    compute similarity between an item c and a set of items I_u. For each item i in I_u, get similarity between 
    i and c, if c exists in the set of items similar to itemid.    
    :param c : itemid of a candidate item
    :param I_u : set of items already purchased by a given user    
    :return w : similarity between c and I_u
    """
    w = 0    
    for iid in I_u :        
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if c in neighbors[iid] :
            w = w + similarities[iid, neighbors[iid] == c][0]    
    return w

##### **2.c- Rank candidate items according to their similarities to $I_u$**

In [ ]:
def rank_candidates(candidates, I_u):
    """
    rank candidate items according to their similarities with i_u    
    :param candidates : list of candidate items
    :param I_u : list of items purchased by the user    
    :return ranked_candidates : dataframe of candidate items, ranked in descending order of similarities with I_u
    """
    
    # list of candidate items mapped to their corresponding similarities to I_u
    sims = [similarity_with_Iu(c, I_u) for c in candidates]
    candidates = iencoder.inverse_transform(candidates)    
    mapping = list(zip(candidates, sims))
    
    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)    
    return ranked_candidates

### Putting all together

Now that we defined all functions necessary to build our item to item top-N recommendation, let's define function ```item2item_topN()``` that makes top-$N$ recommendations for a given user 

In [ ]:
def topn_recommendation(userid, N=30):
    """
    Produce top-N recommendation for a given user    
    :param userid : user for which we produce top-N recommendation
    :param n : length of the top-N recommendation list    
    :return topn
    """
    # find candidate items
    I_u, candidates = candidate_items(userid)
    
    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates(candidates, I_u)
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid','similarity_with_Iu'])    
    topn = pd.merge(topn, movies, on='itemid', how='inner')    
    return topn

In [ ]:
topn_recommendation(test_user)

,itemid,similarity_with_Iu,title
0,1356,52.867173,Ed's Next Move (1996)
1,1189,50.362199,Prefontaine (1997)
2,1516,31.133267,"Wedding Gift, The (1994)"
3,1550,31.031738,Destiny Turns on the Radio (1995)
4,1554,27.364494,Safe Passage (1994)
5,1600,27.287712,Guantanamera (1994)
6,1223,26.631850,King of the Hill (1993)
7,1388,26.624397,Gabbeh (1996)
8,766,26.590175,Man of the Year (1995)
9,691,26.461802,Dark City (1998)


This dataframe represents the top 30 recommendation list for the test user. These items are sorted in decreasing order of similarities with $I_u$.

**Observation** : The recommended items are the most similar to the set $I_u$ of items already purchased by the user.

### Top N recommendation with predictions

Before recommending the previous list to the user, we can go further and predict the ratings the user would have given to each of these items, sort them in descending order of prediction and return the reordered list as the new top N recommendation list.

#### **Rating prediction**


In [ ]:
def predict(userid, itemid):
    """
    Make rating prediction for user userid on item itemid    
    :param userid : id of the active user
    :param itemid : id of the item for which we are making prediction        
    :return r_hat : predicted rating
    """
    
    # Get items similar to item itemid with their corresponding similarities
    item_neighbors = neighbors[itemid]
    item_similarities = similarities[itemid]
    
    # get ratings of user with id userid
    uratings = np_ratings[np_ratings[:, 0].astype('int') == userid]
    
    # similar items rated by item the user of i
    siru = uratings[np.isin(uratings[:, 1], item_neighbors)]
    scores = siru[:, 2]
    indexes = [np.where(item_neighbors == iid)[0][0] for iid in siru[:,1].astype('int')]    
    sims = item_similarities[indexes]
    
    dot = np.dot(scores, sims)
    som = np.sum(np.abs(sims))

    if dot == 0 or som == 0:
        return mean[userid]
    
    return dot / som

Now let's use our ```predict()``` function to predict what ratings the user would have given to the previous top-$N$ list and return the reorganised list (in decreasing order of predictions) as the new top-$N$ list

In [ ]:
def topn_prediction(userid):
    """
    :param userid : id of the active user    
    :return topn : initial topN recommendations returned by the function item2item_topN
    :return topn_predict : topN recommendations reordered according to rating predictions
    """
    # make top N recommendation for the active user
    topn = topn_recommendation(userid)
    
    # get list of items of the top N list
    itemids = topn.itemid.to_list()
    
    predictions = []
    
    # make prediction for each item in the top N list
    for itemid in itemids:
        r = predict(userid, itemid)
        
        predictions.append((itemid,r))
    
    predictions = pd.DataFrame(predictions, columns=['itemid','prediction'])
    
    # merge the predictions to topN_list and rearrange the list according to predictions
    topn_predict = pd.merge(topn, predictions, on='itemid', how='inner')
    topn_predict = topn_predict.sort_values(by=['prediction'], ascending=False)
    
    return topn, topn_predict

Now, let's make recommendation for user 1 and compare the two list

In [ ]:
topn, topn_predict = topn_prediction(userid=test_user)

In [ ]:
topn_predict

,itemid,similarity_with_Iu,title,prediction
7,1388,26.624397,Gabbeh (1996),4.666667
18,359,22.973658,"Assignment, The (1997)",4.600000
4,1554,27.364494,Safe Passage (1994),4.500000
14,1538,24.492453,All Over Me (1997),4.500000
27,1448,20.846909,My Favorite Season (1993),4.490052
29,1375,20.627152,"Cement Garden, The (1993)",4.333333
26,1466,21.063269,Margaret's Museum (1995),4.271915
2,1516,31.133267,"Wedding Gift, The (1994)",4.000000
23,1467,21.861203,"Saint of Fort Washington, The (1993)",4.000000
21,1537,22.061914,Cosi (1996),4.000000


As we noticed, the two lists are sorted in different ways. The second list is organized according to the predictions made for the user.

<b>Note</b>: When making predictions for user $u$ on item $i$, user $u$ may not have rated any of the $k$ most similar items to i. In this case, we consider the mean rating of $u$ as the predicted value.

#### **Evaluation with Mean Absolute Error**

In [ ]:
from recsys.preprocessing import train_test_split, get_examples

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

def evaluate(x_test, y_test):
    print('Evaluate the model on {} test data ...'.format(x_test.shape[0]))
    preds = list(predict(u,i) for (u,i) in x_test)
    mae = np.sum(np.absolute(y_test - np.array(preds))) / x_test.shape[0]
    print('\nMAE :', mae)
    return mae

In [ ]:
evaluate(x_test, y_test)

Evaluate the model on 10000 test data ...

MAE : 0.672389703640273


0.672389703640273

This means that on average, the model's predictions are off by 0.672389703640273